# Chains to file
Subsample all chains

In [1]:
# %load first_cell.py
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import black
import jupyter_black

jupyter_black.load(
    lab=False,
    line_length=79,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)

import numpy as np
import os
import sys
import matplotlib.pyplot as plt

from astropy.cosmology import FlatLambdaCDM

cosmo = FlatLambdaCDM(H0=67.26, Om0=0.314, Ob0=0.049)

from matplotlib import rcParams
rcParams["mathtext.fontset"] = "stix"
rcParams["font.family"] = "STIXGeneral"


def print_shape(*args):
    nelem = len(args)
    for ii in range(nelem - 1):
        print(args[ii].shape, end=" ")
    print(args[nelem - 1].shape)


def ls_level(folder, nlevels):
    for ii in range(nlevels):
        folder = os.path.dirname(folder)
    folder += "/"
    return folder


path_program = ls_level(os.getcwd(), 1)
print(path_program)
sys.path.append(path_program)

DEBUG:jupyter_black:config: {'line_length': 79, 'target_versions': {<TargetVersion.PY310: 10>}}


<IPython.core.display.Javascript object>

/home/jchaves/Proyectos/projects/lya/lya_pk/


In [3]:
from lya_pk.archive import GadgetArchive3D

folder_lya_data = path_program + "data/best_arinyo/"

Archive3D = GadgetArchive3D(
    base_folder=path_program, folder_data=folder_lya_data
)
print(len(Archive3D.training_data))

1650


## Load a chain from central

In [4]:
training_data = Archive3D.training_data

In [59]:
nelem = 500
nparams = 8
out_chain = np.zeros((len(training_data), nelem, nparams))
out_best = np.zeros((len(training_data), nparams))
out_percen = np.zeros((len(training_data), nparams, 3))

In [78]:
# in PIC
folder_chains = "/data/desi/scratch/jchavesm/p3d_fits_new/"
# local
folder_chains = (
    "/home/jchaves/Proyectos/projects/lya/data/pkfits/p3d_fits_new/"
)

for ind_book in range(0, len(training_data)):
    # for ind_book in range(100, 101):
    scale_tau = training_data[ind_book]["val_scaling"]
    ind_z = training_data[ind_book]["z"]
    sim_label = training_data[ind_book]["sim_label"]

    tag = (
        "fit_sim"
        + sim_label[4:]
        + "_tau"
        + str(np.round(scale_tau, 2))
        + "_z"
        + str(ind_z)
        + "_kmax3d"
        + str(Archive3D.kmax_3d)
        + "_noise3d"
        + str(Archive3D.noise_3d)
        + "_kmax1d"
        + str(Archive3D.kmax_1d)
        + "_noise1d"
        + str(Archive3D.noise_1d)
    )
    # check folder is not None
    file_arinyo = np.load(folder_chains + tag + ".npz")
    chain = file_arinyo["chain"].copy()
    chain[:, 0] = -np.abs(chain[:, 0])
    best = file_arinyo["best_params"].copy()
    best[0] = -np.abs(best[0])

    ind = np.random.permutation(chain.shape[0])[:nelem]

    out_chain[ind_book, ...] = chain[ind]
    out_best[ind_book, :] = best
    out_percen[ind_book, ...] = np.percentile(chain, [25, 50, 75], axis=0).T

In [79]:
np.savez(
    "subsamp_chains_mpg_hyp",
    chains=out_chain,
    best=out_best,
    percen_25_50_75=out_percen,
)

In [80]:
res = np.load("subsamp_chains_mpg_hyp.npz")

In [81]:
res.files

['chains', 'best', 'percen_25_50_75']